# Batch Generation

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaForCausalLM

model_id = 'huggyllama/llama-65b'
# model_id = 'meta-llama/Llama-2-7b-hf'
# model_id = 'huggyllama/llama-30b'
# model_id = 'meta-llama/Llama-2-70b-hf'
# model_id = 'meta-llama/Llama-2-13b-hf'
USE_FINETUNED = False
N_SHOT = 3
DATASET = 'MIMIC'
# DATASET = 'OPENI'
BATCH_SZ = 1
DEVICE = 3
lora_r= 2


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
IS_ZEROSHOT = 1 if not USE_FINETUNED and N_SHOT==0 else 0
print(IS_ZEROSHOT)

0


In [3]:
#Load from checkpoint
# checkpoint_name = '/home/workspace/CXRLabler_trained_model/llama2-7b_v1/adapter_model.bin' 
# checkpoint_name = '/home/workspace/CXRLabler_trained_model/llama-30b_v1/adapter_model.bin' 
# checkpoint_name = '/home/workspace/CXRLabler_trained_model/llama2-70b_v1/adapter_model.bin' 
checkpoint_name = '/home/workspace/CXRLabler_trained_model/llama-65b/adapter_model.bin' 

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir='/home/public_models')
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"  # Allow batched inference

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = LlamaForCausalLM.from_pretrained(
    model_id, 
    quantization_config=bnb_config, 
    # device_map='auto',
    device_map={"":DEVICE},
    cache_dir='/home/public_models'
)

Loading checkpoint shards: 100%|██████████| 14/14 [15:35<00:00, 66.80s/it]


In [5]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    # cls = bnb.nn.Linear4bit if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [6]:
if USE_FINETUNED:
    from peft import (
        LoraConfig,
        get_peft_model,
        get_peft_model_state_dict,
        prepare_model_for_kbit_training,
        set_peft_model_state_dict,
    )
    lora_target_modules= find_all_linear_names(model)
    # lora_target_modules= [
    #     "q_proj",
    #     "v_proj",
    #     # "k_proj",
    #     # "o_proj",
    # ]

    config = LoraConfig(
        r=lora_r,
        lora_alpha=16,
        target_modules=lora_target_modules,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)
    set_peft_model_state_dict(model, torch.load(checkpoint_name,  map_location=torch.device(f"cuda:{DEVICE}")))

In [7]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [8]:
# #Load from saved
# from peft import PeftModel
# model = PeftModel.from_pretrained(model, '/lora-alpaca-MG-30b/')

In [9]:
import pandas as pd

if DATASET == 'MIMIC':
    data_path = '/home/data/CXR_Labeler/mimic_test_500_answer.csv'
    data = pd.read_csv(data_path)
elif DATASET == 'OPENI':
    data_path = '/home/data/CXR_Labeler/openi_test_500_answer (1).csv'
    data = pd.read_csv(data_path)

print(DATASET)

MIMIC


In [10]:
import json
import random
random.seed(1125)

train_path = '/home/data/CXR_Labeler/CXR_training_data_v3.json'
with open(train_path, "r", encoding="utf-8-sig") as f:
    train = json.load(f)

In [11]:
# instruction = train[0]['instruction']
# bars='-'*10
# if USE_FINETUNED:
#     prompt = [f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input: {f}\n\n### Response:" for f in data['Findings']]
# elif N_SHOT>0: 
#     instruction = instruction[:-2] + f" like {N_SHOT} examples below."

#     prompt = []
#     for f in data['Findings']:
#         eg = random.sample(train, k=N_SHOT)
#         examples = ""
#         for k in range(N_SHOT):
#             text = f"""Example{k+1}:\n### Input: {eg[k]['input']}\n### Response: {eg[k]['output']}"""
#             examples += text + "\n"

#         pt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:{instruction}\n\n{bars}\n{examples}{bars}\n\nHere is the report that I want you to extract lesion information.\n### Input: {f}\n### Response:"""
#         prompt.append(pt)
# else:
#     # zero-shot prompt
#     prompt = []
#     for f in data['Findings']:
#         pt = """As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. 
# Default to 'Undefined' if the finding is not explicily mentioned. If a finding is explicily mentioned as present, mark 'Yes'. If a finding is explicily mentioned as absent, mark 'No'. If a finding is explicily mentioned but unclear, mark 'Maybe'. 

# Template:
# {
# 'Atelectasis': '[ANSWER]',
# 'Cardiomegaly': '[ANSWER]',
# 'Consolidation': '[ANSWER]',
# 'Edema': '[ANSWER]',
# 'Enlarged Cardiomediastinum': '[ANSWER]',
# 'Fracture': '[ANSWER]',
# 'Lung Lesion': '[ANSWER]',
# 'Lung Opacity': '[ANSWER]',
# 'Pleural Effusion': '[ANSWER]',
# 'Pleural Other': '[ANSWER]',
# 'Pneumonia': '[ANSWER]',
# 'Pneumothorax': '[ANSWER]',
# 'Support Devices': '[ANSWER]',
# }

# Report:
# """ + f + """

# Answer according to the template:
# {
# 'Atelectasis': '"""
#         prompt.append(pt)

In [12]:
instruction = train[0]['instruction']
bars='-'*10
if USE_FINETUNED:
    prompt = [f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input: {f}\n\n### Response:" for f in data['Findings']]
elif N_SHOT>0: 
    import ast
    import json

    # 원하는 순서로 키 매핑
    key_mapping = {
        'A': 'Atelectasis',
        'Ca': 'Cardiomegaly',
        'LL': 'Lung Lesion',
        'LO': 'Lung Opacity',
        'E': 'Edema',
        'Co': 'Consolidation',
        'P': 'Pneumonia',
        'Px': 'Pneumothorax',
        'PE': 'Pleural Effusion',
        'PO': 'Pleural Other',
        'F': 'Fracture',
        'SD': 'Support Devices',
        'EC': 'Enlarged Cardiomediastinum'
    }

    # 변환 함수 정의
    def transform_output(output_str):
        # 문자열을 딕셔너리로 변환
        output_dict = ast.literal_eval(output_str)
        new_output = {}
        for key in key_mapping:
            if key in output_dict:
                new_output[key_mapping[key]] = output_dict[key]
        # 딕셔너리를 문자열로 변환
        dict_output = json.dumps(new_output)
        dict_output = dict_output.replace('-1', 'Maybe')
        dict_output = dict_output.replace('1', 'Yes')
        dict_output = dict_output.replace('0', 'No')
        dict_output = dict_output.replace('-2', 'Undefined')
        return dict_output

    prompt = []
    for f in data['Findings']:
        # N_SHOT개의 무작위 예시 생성
        eg = random.sample(train, k=N_SHOT)
        eg_transformed = [transform_output(eg[k]['output']) for k in range(N_SHOT)]
        examples = ""
        for k in range(N_SHOT):
            text = f"""Example{k+1}:\nReport: {eg[k]['input']}\nAnswer according to the template: {eg_transformed[k]}"""
            examples += text + "\n"

        # few-shot 프롬프트 구성
        pt = f"""As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. 
Default to 'Undefined' if the finding is not explicitly mentioned. If a finding is explicitly mentioned as present, mark 'Yes'. If a finding is explicitly mentioned as absent, mark 'No'. If a finding is explicitly mentioned but unclear, mark 'Maybe'. 

Template:
{{"Atelectasis": "[ANSWER]", "Cardiomegaly": "[ANSWER]", "Consolidation": "[ANSWER]", "Edema": "[ANSWER]", "Enlarged Cardiomediastinum": "[ANSWER]", "Fracture": "[ANSWER]", "Lung Lesion": "[ANSWER]", "Lung Opacity": "[ANSWER]", "Pleural Effusion": "[ANSWER]", "Pleural Other": "[ANSWER]", "Pneumonia": "[ANSWER]", "Pneumothorax": "[ANSWER]", "Support Devices": "[ANSWER]"}}

{examples}

Report: {f}
Answer according to the template: {{"Atelectasis": """
        prompt.append(pt)

else:
    # zero-shot prompt
    prompt = []
    for f in data['Findings']:
        pt = """As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. 
Default to 'Undefined' if the finding is not explicily mentioned. If a finding is explicily mentioned as present, mark 'Yes'. If a finding is explicily mentioned as absent, mark 'No'. If a finding is explicily mentioned but unclear, mark 'Maybe'. 

Template:
{
'Atelectasis': '[ANSWER]',
'Cardiomegaly': '[ANSWER]',
'Consolidation': '[ANSWER]',
'Edema': '[ANSWER]',
'Enlarged Cardiomediastinum': '[ANSWER]',
'Fracture': '[ANSWER]',
'Lung Lesion': '[ANSWER]',
'Lung Opacity': '[ANSWER]',
'Pleural Effusion': '[ANSWER]',
'Pleural Other': '[ANSWER]',
'Pneumonia': '[ANSWER]',
'Pneumothorax': '[ANSWER]',
'Support Devices': '[ANSWER]',
}

Report:
""" + f + """

Answer according to the template:
{
'Atelectasis': '"""
        prompt.append(pt)

In [13]:
def post_process_llama_label(text, is_zeroshot=False):
    if USE_FINETUNED:
        text = text.split('### Response:')[-1]
    else:
        text = text.split('Answer according to the template:')[-1]
        text = text.split("}")[0]+'}'
        text = text.replace('\n','')
    return text.strip()

In [14]:
def generate_label(model, batch, is_zeroshot=False):
    max_new_tokens = 80
    if not USE_FINETUNED:
        max_new_tokens = 150

    with torch.no_grad():
        generated_ids = model.generate(
                            **tokenizer(batch, 
                                return_tensors='pt', 
                                return_token_type_ids=False, 
                                truncation=False, 
                                padding=True, 
                            ).to('cuda'),
                            max_new_tokens=max_new_tokens,
                            early_stopping=True,
                            do_sample=False,
                            eos_token_id=2,
                            )
        
    generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    print(generated_texts)
    generated_texts = [post_process_llama_label(t, is_zeroshot) for t in generated_texts]
    return generated_texts

In [15]:
def divide_batch(l:list, batch_size):
    for i in range(0, len(l), batch_size): 
        yield l[i:i + batch_size]

In [16]:
batchs = list(divide_batch(prompt, batch_size=BATCH_SZ)) 

In [17]:
from tqdm import tqdm
labels = []
for batch in tqdm(batchs):
    label = generate_label(model, batch, IS_ZEROSHOT)
    labels.append(label)
    
labels = sum(labels, [])

  0%|          | 0/500 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
  0%|          | 1/500 [01:02<8:41:52, 62.75s/it]

['As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. \nDefault to \'Undefined\' if the finding is not explicitly mentioned. If a finding is explicitly mentioned as present, mark \'Yes\'. If a finding is explicitly mentioned as absent, mark \'No\'. If a finding is explicitly mentioned but unclear, mark \'Maybe\'. \n\nTemplate:\n{"Atelectasis": "[ANSWER]", "Cardiomegaly": "[ANSWER]", "Consolidation": "[ANSWER]", "Edema": "[ANSWER]", "Enlarged Cardiomediastinum": "[ANSWER]", "Fracture": "[ANSWER]", "Lung Lesion": "[ANSWER]", "Lung Opacity": "[ANSWER]", "Pleural Effusion": "[ANSWER]", "Pleural Other": "[ANSWER]", "Pneumonia": "[ANSWER]", "Pneumothorax": "[ANSWER]", "Support Devices": "[ANSWER]"}\n\nExample1:\nReport: Linear areas of atelectasis/ scarring are seen in the right mid lung and lateral left lung base. Patchy opacity seen posteriorly on the lateral view may

  0%|          | 2/500 [01:43<6:52:05, 49.65s/it]

['As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. \nDefault to \'Undefined\' if the finding is not explicitly mentioned. If a finding is explicitly mentioned as present, mark \'Yes\'. If a finding is explicitly mentioned as absent, mark \'No\'. If a finding is explicitly mentioned but unclear, mark \'Maybe\'. \n\nTemplate:\n{"Atelectasis": "[ANSWER]", "Cardiomegaly": "[ANSWER]", "Consolidation": "[ANSWER]", "Edema": "[ANSWER]", "Enlarged Cardiomediastinum": "[ANSWER]", "Fracture": "[ANSWER]", "Lung Lesion": "[ANSWER]", "Lung Opacity": "[ANSWER]", "Pleural Effusion": "[ANSWER]", "Pleural Other": "[ANSWER]", "Pneumonia": "[ANSWER]", "Pneumothorax": "[ANSWER]", "Support Devices": "[ANSWER]"}\n\nExample1:\nReport: An orogastric tube courses below the diaphragm, the tip projects over the gastric body. Lung volumes are decreased, accentuating the cardiac silhouette.

  1%|          | 3/500 [02:25<6:22:09, 46.14s/it]

...

['As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. \nDefault to \'Undefined\' if the finding is not explicitly mentioned. If a finding is explicitly mentioned as present, mark \'Yes\'. If a finding is explicitly mentioned as absent, mark \'No\'. If a finding is explicitly mentioned but unclear, mark \'Maybe\'. \n\nTemplate:\n{"Atelectasis": "[ANSWER]", "Cardiomegaly": "[ANSWER]", "Consolidation": "[ANSWER]", "Edema": "[ANSWER]", "Enlarged Cardiomediastinum": "[ANSWER]", "Fracture": "[ANSWER]", "Lung Lesion": "[ANSWER]", "Lung Opacity": "[ANSWER]", "Pleural Effusion": "[ANSWER]", "Pleural Other": "[ANSWER]", "Pneumonia": "[ANSWER]", "Pneumothorax": "[ANSWER]", "Support Devices": "[ANSWER]"}\n\nExample1:\nReport: Frontal and lateral views of the chest. No prior. The lungs are clear. Costophrenic angles are sharp. Cardiomediastinal silhouette is within normal limi

100%|█████████▉| 499/500 [5:53:21<00:42, 42.74s/it]

['As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. \nDefault to \'Undefined\' if the finding is not explicitly mentioned. If a finding is explicitly mentioned as present, mark \'Yes\'. If a finding is explicitly mentioned as absent, mark \'No\'. If a finding is explicitly mentioned but unclear, mark \'Maybe\'. \n\nTemplate:\n{"Atelectasis": "[ANSWER]", "Cardiomegaly": "[ANSWER]", "Consolidation": "[ANSWER]", "Edema": "[ANSWER]", "Enlarged Cardiomediastinum": "[ANSWER]", "Fracture": "[ANSWER]", "Lung Lesion": "[ANSWER]", "Lung Opacity": "[ANSWER]", "Pleural Effusion": "[ANSWER]", "Pleural Other": "[ANSWER]", "Pneumonia": "[ANSWER]", "Pneumothorax": "[ANSWER]", "Support Devices": "[ANSWER]"}\n\nExample1:\nReport: As compared to prior chest radiograph from ___, there has been interval removal of a right-sided pigtail catheter. Miniscule collection of right apical 

100%|██████████| 500/500 [5:54:04<00:00, 42.49s/it]

['As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. \nDefault to \'Undefined\' if the finding is not explicitly mentioned. If a finding is explicitly mentioned as present, mark \'Yes\'. If a finding is explicitly mentioned as absent, mark \'No\'. If a finding is explicitly mentioned but unclear, mark \'Maybe\'. \n\nTemplate:\n{"Atelectasis": "[ANSWER]", "Cardiomegaly": "[ANSWER]", "Consolidation": "[ANSWER]", "Edema": "[ANSWER]", "Enlarged Cardiomediastinum": "[ANSWER]", "Fracture": "[ANSWER]", "Lung Lesion": "[ANSWER]", "Lung Opacity": "[ANSWER]", "Pleural Effusion": "[ANSWER]", "Pleural Other": "[ANSWER]", "Pneumonia": "[ANSWER]", "Pneumothorax": "[ANSWER]", "Support Devices": "[ANSWER]"}\n\nExample1:\nReport: PA and lateral views of the chest provided. Lung volumes are low limiting assessment. Allowing for this, the lungs are clear without focal consolidation

In [24]:
len(labels)

500

In [19]:
1

1

In [25]:
import os
inference_result_path = "/home/workspace/inference_result/CXR"
if USE_FINETUNED:
    file_name = checkpoint_name.split("/")[4] + f"-{DATASET}.csv"
else:
    file_name = model_id.split('/')[-1] + f"_NSHOT{N_SHOT}-{DATASET}_4.csv"

data['result']= labels
data.to_csv(os.path.join(inference_result_path, file_name), index=False)
print("saved at ", inference_result_path+'/'+file_name)

saved at  /home/workspace/inference_result/CXR/llama-65b_NSHOT3-MIMIC_4.csv


In [26]:
import ast 
# 예외가 발생한 행의 인덱스를 저장할 리스트
error_indices = []

# 딕셔너리로 변환된 데이터를 저장할 리스트
dicts = []

for i, item in enumerate(data['result']):
    try:
        # 문자열을 딕셔너리로 변환
        dicts.append(ast.literal_eval(item))
    except ValueError:
        # 변환 중 오류가 발생한 경우, 인덱스 저장
        error_indices.append(i)

# 예외가 발생하지 않은 행들로부터 데이터프레임 생성
df = pd.DataFrame(dicts)

# 오류가 발생한 행의 인덱스 출력
print("오류가 발생한 행의 인덱스:", error_indices)

# 데이터프레임 결과 확인
print(df)

오류가 발생한 행의 인덱스: []
    Atelectasis Cardiomegaly Lung Lesion Lung Opacity      Edema  \
0           Yes           No   Undefined    Undefined         No   
1     Undefined           No   Undefined          Yes         No   
2     Undefined    Undefined         Yes          Yes  Undefined   
3     Undefined          Yes   Undefined    Undefined  Undefined   
4           Yes        Maybe   Undefined    Undefined  Undefined   
..          ...          ...         ...          ...        ...   
495   Undefined    Undefined   Undefined           No  Undefined   
496         Yes           No         Yes          Yes  Undefined   
497   Undefined    Undefined   Undefined    Undefined  Undefined   
498   Undefined           No   Undefined    Undefined         No   
499   Undefined           No   Undefined    Undefined         No   

    Consolidation  Pneumonia Pneumothorax Pleural Effusion Pleural Other  \
0              No  Undefined           No               No     Undefined   
1       Unde

In [27]:
df = df.replace('-2', '2')
df = df.replace('No', '0')
df = df.replace('Yes', '1')
df = df.replace('Maybe', '-1')
df = df.replace('Undefined', '2')

column_order = [
    'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion',
    'Lung Opacity', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
    'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
    'Support Devices'
]
df = df[column_order]

df.to_csv(os.path.join(inference_result_path, 'dabin_'+file_name), index=False)